<img src = "./resources/images/header_banner_2.jpeg" alt = "Encabezado MLDS" width = "100%"></img>

# **Entendimiento de los Datos**
---

## **0. Integrantes del equipo de trabajo**
---

<table><thead>
  <tr>
    <th>#</th>
    <th>Integrante</th>
    <th>Documento de identidad</th>
  </tr></thead>
<tbody>
  <tr>
    <td>1</td>
    <td>Laura Alejandra Díaz López</td>
    <td>1010018062</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Diego Alejandro Feliciano Ramos</td>
    <td>1024586904</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Geraldine Gracia Ruiz</td>
    <td>1032488268</td>
  </tr>
</tbody>
</table>

## **1. Carga de datos**
---
Dado que, en la entrega anterior se hicieron cambios que permitieron subir el conjunto de datos a un repositorio en GitHub se trabajará desde este mismo espacio. Por tanto se hará la descripción del proceso de descarga y carga de los datos al notebook: 

In [ ]:
# Instalamos los paquetes necesarios para el notebook en caso de trabajar desde Google Colab
!pip install wget pandas ydata-profiling ipywidgets

In [ ]:
import wget

# Descargar el dataset del repositorio publico
url = 'https://github.com/MLDS-UN-ProjectTeam/final-term-project/raw/main/resources/data/Saber_11_2023-2.zip?download='
downloaded_file_name = 'Saber 11 2023-2.zip'
wget.download(url = url, out = downloaded_file_name)
decompressed_file_name = downloaded_file_name.replace('zip', 'csv')

In [ ]:
import zipfile

# Se descomprime el archivo en modo de lectura
with zipfile.ZipFile(downloaded_file_name, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import pandas as pd
# Cargar en el notebook el conjunto de datos con las restricciones conocidas:
# - Delimitador: ¬ 
# - Quoting = 3 => Sin comillas para todas las variables
saber11_dataframe = pd.read_csv(decompressed_file_name, delimiter='¬', quoting=3)

## **2. Análisis Exploratorio de los Datos**
---

### **2.1. Resumen General**
---

En nuestro _overview_ del dataset pudimos registrar los siguientes datos:
- Formato: .csv
- Cantidad de registros: 551149
- Tamaño del archivo en MB comprimido: 71,3 MB
- Tamaño del archivo en MB descomprimido: 448 MB
- Tamaño del _dataset_ en memoria: 349 MB
- Tamaño completo del _dataset_ en memoria (con uso de memoria profundo): 2.0 GB

In [ ]:
# Revisamos la salud del dataset cargado cargando los primeros 5 elementos del mismo
saber11_dataframe.head()

In [ ]:
# Obtenemos un esbozo sobre la información del dataset y las columnas del mismo 
saber11_dataframe.info(memory_usage='deep')

### **2.2. Análisis estadístico descriptivo**
---

Describa el conjunto de datos por medio de estadística descriptiva, aportando interpretaciones precisas que ayuden a entender los datos.


In [ ]:
from ydata_profiling import ProfileReport
# Se genera un archivo HTML para poder verlo de forma separada y no cargar el notebook con información innecesaria
profile = ProfileReport(saber11_dataframe)
profile.to_file('Saber 11 2023-2-Report.html')
# De cualquier forma este reporte se puede ver en el notebook con alguna de las siguientes líneas de código comentadas (no recomendable, es gigantesco)
# profile.to_notebook_iframe()
# profile.to_widgets()

Algo importante a notar y tener en cuenta para este análisis estadístico descriptivo de los datos, es que se cuenta con pocas variables numéricas y, en cambio, varias de tipo categórico, por lo que se dificulta parte del uso de las herramientas disponibles.

Se recurrirá entonces a la estrategía de convertir algunas variables de interés de categóricas a numéricas al reemplazar valores simples como "sí" o "no", por 1 o 0.

In [ ]:
# TRANSFORMACIÓN DE ALGUNAS VARIABLES CATEGÓRICAS DE INTERÉS.

# GENERO
saber11_dataframe['ESTU_GENERO_N'] = saber11_dataframe['ESTU_GENERO'].map({'M': 0, 'F': 1})

# 14 ESTRATO
rangos_estrato = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0
}
saber11_dataframe['FAMI_ESTRATOVIVIENDA_N'] = saber11_dataframe['FAMI_ESTRATOVIVIENDA'].map(rangos_estrato)


# 15 PERSONAS QUE VIVEN EN EL HOGAR
rangos_pers_hog = {
    '1 a 2': 1.5,
    '3 a 4': 3.5,
    '5 a 6': 5.5,
    '7 a 8': 7.5,
    '9 o más': 9
}
saber11_dataframe['FAMI_PERSONASHOGAR_N'] = saber11_dataframe['FAMI_PERSONASHOGAR'].map(rangos_pers_hog)

# TIENE INTERNET
rangos_si_no = {
    'Si': 1,
    'No': 0
}
saber11_dataframe['FAMI_TIENEINTERNET_N'] = saber11_dataframe['FAMI_TIENEINTERNET'].map(rangos_si_no)

# TIENE COMPUTADOR
saber11_dataframe['FAMI_TIENECOMPUTADOR_N'] = saber11_dataframe['FAMI_TIENECOMPUTADOR'].map(rangos_si_no)

# NÚMERO DE LIBROS EN LA FAMILIA
saber11_dataframe['FAMI_NUMLIBROS']
rangos_libros = {
    '0 A 10 LIBROS' : 5,
    '11 A 25 LIBROS' : 18,
    '26 A 100 LIBROS' : 63,
    'MÁS DE 100 LIBROS' : 150
}
saber11_dataframe['FAMI_NUMLIBROS_N'] = saber11_dataframe['FAMI_NUMLIBROS'].map(rangos_libros)

# HORAS QUE TRABAJA SEMANALMENTE
rangos_hor_tra = {
    'Menos de 10 horas' : 5,
    'Entre 11 y 20 horas' : 16,
    'Entre 21 y 30 horas' : 26,
    'Más de 30 horas' : 35
}
saber11_dataframe['ESTU_HORASSEMANATRABAJA_N'] = saber11_dataframe['ESTU_HORASSEMANATRABAJA'].map(rangos_hor_tra)

#ESTUVO PRIVADO DE LA LIBERTAD
rangos_n_s= {
    'S' : 1,
    'N' : 0
}
saber11_dataframe['ESTU_PRIVADO_LIBERTAD_N'] = saber11_dataframe['ESTU_PRIVADO_LIBERTAD'].map(rangos_n_s)

Ahora que se han convertido algunas de las variables de interés, se podrá proceder a hacer un análisis estadístico de ellas. Cabe notar que para hacer esta transformación fue necesario hacer varias estimaciones, pues varios de los datos son dados en rangos del estilo 'de 1 a 3 días', casos en los cuales se tomó la media.

In [ ]:
saber11_dataframe[['FAMI_ESTRATOVIVIENDA_N', 'ESTU_GENERO_N',	'FAMI_PERSONASHOGAR_N',	'FAMI_TIENEINTERNET_N',	'FAMI_TIENECOMPUTADOR_N',	'FAMI_NUMLIBROS_N',	'ESTU_HORASSEMANATRABAJA_N',	'ESTU_PRIVADO_LIBERTAD_N']].describe()



<img src = "./resources/images/categoricas_mod.png" alt = "Parámetros de estadística descriptiva evaluados sobre las variables categóricas ya transformadas" width = "100%"></img>


Con los datos obtenidos gracias al método 'describe' es posible llegar a varias conclusiones, respecto a las variables de tipo socioecónomico:

- Para las variables categóricas de interés donde las posibles respuestas son solo dos se realizó una conversión donde el valor puede ser solo 0 o 1. La mayoría presenta una media de 0.5 junto a una desviación estándar similar, lo que indica que los datos están muy bien distribuídos entre las dos opciones, un parámetro importante a tener en cuenta.
- La mayoría de variables de todo tipo presenta una desviación estándar aceptable considerando la media, el mínimo y el máximo, lo cual indica que se tiene un buen conjunto de datos con el cual trabajar.

Por supuesto, otras de las variables más importantes a analizar son los resultados del examen.

In [ ]:
saber11_dataframe[['PUNT_LECTURA_CRITICA', 'PUNT_MATEMATICAS', 'PUNT_C_NATURALES', 'PUNT_SOCIALES_CIUDADANAS', 'PUNT_INGLES', 'PUNT_GLOBAL']].describe()

<img src = "./resources/images/puntajes_des.png" alt = "Parámetros de estadística descriptiva evaluados sobre los resultados del examen" width = "100%"></img>

Al observar la descripción estadística de los puntajes del examen, es interesante ver como todas las componentes del mismo (matemáticas, lectura crítica, etc.) tienen tanto un promedio como una desviación estándar similares, dando a enteder que es fiable trabajar directamente con el puntaje global.

Y hablando particularmente del puntaje global, al tener un mínimo de 0, un máximo de 500, una media de aproximadamente 253 y una desviación estandar de 51.3 es clara la forma en que la mayoría de datos se encuentran cerca a la media, por lo que casos de muy buen o muy mal puntaje en el examen tendrán que verse como casos extraordinarios que vale la pena estudiar.

### **2.3. Resumen de la Calidad de los datos**
---

- ¿Se detecta la ausencia de datos, la presencia de datos erróneos o la existencia de datos de baja calidad en el conjunto?
- ¿Se identifican registros con datos ilegibles o con dificultades de codificación durante la revisión del conjunto de datos?
- ¿Se observa una diversidad de formatos en el conjunto de datos que pueda dificultar su consistencia o comprensión?
- ¿Se identificaron y abordaron posibles problemas, como valores atípicos, duplicados o datos faltantes?

En el análisis se ha identificado la ausencia de datos en 37 variables del conjunto de datos. A continuación, se presentan cuatro tablas que muestran la relación entre cada variable y el porcentaje de datos faltantes, proporcionando una visión clara de la distribución de estos valores ausentes en las columnas.

**1. Variables con más del 90% de datos faltantes:**

Se observaron dos variables, ESTU_LENGUANATIVA y PERCENTIL_ESPECIAL_GLOBAL, que exhiben un porcentaje superior al 90% de datos ausentes. Estas variables se eliminarán en la etapa de selección de características.

| Variable                      | Porcentaje de datos faltantes (%) |
| ----------------------------- | --------------------------------- |
| ESTU_LENGUANATIVA             | 98.311                            |
| PERCENTIL_ESPECIAL_GLOBAL     | 92.112                            |

**2. Variable con 18.55% de datos faltantes:**

La variable COLE_BILINGUE presenta un 18.55% de datos faltantes. Aunque este porcentaje es menor en comparación con el correspondiente al de valores faltantes en las variables anteriores, aún puede influir en la calidad del análisis. Se evaluará su utilidad en el análisis en la siguiente etapa.

| Variable                      | Porcentaje de datos faltantes (%) |
| ----------------------------- | --------------------------------- |
| COLE_BILINGUE                 | 18.552                            |

**3. Variables con entre un 5% y 10% de datos faltantes:**

Un total de veintiséis variables contienen entre un 5% y un 10% de datos faltantes. Este rango sugiere una falta moderada de información, esto puede afectar la calidad de nuestro análisis y conclusiones. Más adelante se discutirán las posibles estrategias para abordar este problema.

| Variable                      | Porcentaje de datos faltantes (%) |
| ----------------------------- | --------------------------------- |
| FAMI_COMELECHEDERIVADOS       | 9.181                             |
| FAMI_COMECEREALFRUTOSLEGUMBRE | 8.933                             |
| FAMI_ESTRATOVIVIENDA          | 8.910                             |
| FAMI_NUMLIBROS                | 8.799                             |
| ESTU_DEDICACIONINTERNET       | 8.362                             |
| FAMI_COMECARNEPESCADOHUEVO    | 8.315                             |
| FAMI_TIENESERVICIOTV          | 8.198                             |
| ESTU_DEDICACIONLECTURADIARIA  | 8.147                             |
| FAMI_TIENEINTERNET            | 7.967                             |
| FAMI_EDUCACIONMADRE           | 7.884                             |
| FAMI_EDUCACIONPADRE           | 7.834                             |
| FAMI_SITUACIONECONOMICA       | 5.974                             |
| FAMI_TIENECONSOLAVIDEOJUEGOS  | 5.697                             |
| ESTU_TIPOREMUNERACION         | 5.665                             |
| FAMI_TIENEAUTOMOVIL           | 5.659                             |
| FAMI_TIENEHORNOMICROOGAS      | 5.579                             |
| FAMI_TIENEMOTOCICLETA         | 5.556                             |
| FAMI_TRABAJOLABORPADRE        | 5.554                             |
| ESTU_INSE_INDIVIDUAL          | 5.525                             |
| ESTU_NSE_INDIVIDUAL           | 5.525                             |
| ESTU_HORASSEMANATRABAJA       | 5.518                             |
| FAMI_CUARTOSHOGAR             | 5.504                             |
| FAMI_TRABAJOLABORMADRE        | 5.499                             |
| FAMI_TIENECOMPUTADOR          | 5.470                             |
| FAMI_TIENELAVADORA            | 5.469                             |
| FAMI_PERSONASHOGAR            | 5.290                             |

**4. Variables con menos del 5% de datos faltantes:**

Ocho variables tienen menos del 5% de valores faltantes, indicando una mejor cobertura de información en estas columnas. Sin embargo, es importante evaluarlas y definir cómo se manejarán para reducir el impacto en la calidad del análisis.

| Variable                      | Porcentaje de datos faltantes (%) |
| ----------------------------- | --------------------------------- |
| COLE_CARACTER                 | 4.100                             |
| PERCENTIL_GLOBAL              | 0.453                             |
| PUNT_INGLES                   | 0.453                             |
| PERCENTIL_INGLES              | 0.453                             |
| DESEMP_INGLES                 | 0.453                             |
| ESTU_PRESENTACIONSABADO       | 0.076                             |
| ESTU_NSE_ESTABLECIMIENTO      | 0.012                             |
| ESTU_GENERO                   | 0.001                             |

En cuanto a datos erróneos, no se encontraron errores el conjunto de datos.

Además, se han identificado cuatro variables que se consideran no informativas, pues sus columnas presentan un valor constante en todos los registros. Esta homogeneidad en los datos sugiere que no aportan variabilidad y, por tanto, podrían no contribuir significativamente al análisis. Estas variables se omitirán en etapas posteriores.

| Variable                 | Cantidad de valores distintos/únicos | Valor constante |
| ------------------------ | ------------------------------------ | --------------- |
| PERIODO                  | 1                                    | 20234           |
| ESTU_ESTUDIANTE          | 1                                    | ESTUDIANTE      |
| ESTU_GENERACION          | 1                                    | NO              |
| ESTU_ESTADOINVESTIGACION | 1                                    | PUBLICAR        |


No se han encontrado registros con datos ilegibles ni con dificultades de codificación. El formato del conjunto de datos es uniforme y no presenta inconsistencias que puedan afectar la comprensión general. Sin embargo, se ha observado la presencia de variables redundantes. Esta redundancia en las variables, combinada con el elevado número total de variables, puede dificultar el proceso de análisis antes de llevar a cabo una limpieza del conjunto de datos.

En esta fase del proyecto, se identificaron problemas potenciales en el conjunto de datos, entre ellos datos faltantes, sin embargo, aún no se han abordado. En la siguiente etapa, se llevará a cabo la selección de variables y se reducirá la información a una parte de la población (una sola ciudad), lo que permitirá una evaluación más precisa de cómo manejar los datos faltantes. Se explorarán estrategias adecuadas como la imputación por medidas de tendencia central o la eliminación de registros donde esto no altere de manera significativa al conjunto de datos. Este enfoque garantizará un manejo efectivo de este problema y mejorará la calidad de la información disponible para un análisis más preciso.

### **2.4. Tipos de variables**
---

En nuestro trabajo se tiene como objetivo estimar los resultados de las pruebas saber 11 basados en determinadas variables del set de datos, particularmente las variables socioeconómicas, así, se entiende que el puntaje total del examen de estado es la variable a estimar. 
Esta variable anteriormente nombrada es continua en un intervalo de 0 a 500.  En resumen:

| Variable                 	 | Tipo     	 | Rango   	 |
|----------------------------|------------|-----------|
| PUNT_GLOBAL              	 | Continua 	 | 0 - 500 	 |

Ahora, para el análisis de etiquetas y desbalanceo nos apoyamos del _profile_ generado anteriormente, en cuyo caso tenemos los siguientes resultados:

<img src = "./resources/images/imbalance.png" alt = "Desbalanceo del dataset" width = "100%"></img>

Así, se empieza a hacer el análisis de la primera variable con problemas de balanceo en las etiquetas, en este caso es el tipo de documento con el que se presentó el estudiante a realizar el examen de estado. En el profile obtenido se puede evaluar que hay una fuerte tendencia a que los estudiantes presenten el examen con tarjeta de identidad (TI), con una prevalencia del 85 % y que el segundo tipo de documento con mayor prevalencia es la cédula de ciudadanía con un 12.8 %. Esto deja ver que al menos el 97.8% de los evaluados cuentan con nacionalidad colombiana y que, por tanto, esta variable puede no ser adecuada para ser usada en un modelo posteriormente.

<img src = "./resources/images/imbalance_estu_tipodocumento.png" alt = "Desbalanceo de ESTU_TIPODOCUMENTO" width = "100%"></img>

Siguiendo por la línea del anterior análisis nos podemos dar cuenta que la conclusión anterior, aunque apresurada, fue correcta y es que el 98.5 % de las ocurrencias en esta categoría indican que la nacionalidad del estudiante es colombiana y que la segunda mayor nacionalidad es venezolana con un 1.5 % aproximadamente, dejando menos del 0.1 % de ocurrencias distribuídas entre otras 37 nacionalidades. Por tanto, esta variable también se entiende que puede no ser adecuada para su posible uso en un modelo en el futuro.

<img src = "./resources/images/imbalance_estu_nacionalidad.png" alt = "Desbalanceo de ESTU_NACIONALIDAD" width = "100%"></img>

De nuevo, analizando el desbalanceo en la variable de país de residencia del estudiante, se ve una tendencia similar y posiblemente una alta correlación entre la nacionalidad y el país de residencia; alcanzando Colombia, como país de residencia un 98,5 % del total de las ocurrencias seguido, de igual forma, por Venezuela con un 1.5 % aproximadamente. De aquí se concluye que así como con las dos variables anteriores esta no sería de mucha utilidad para su uso en algún modelo futuro.

<img src = "./resources/images/imbalance_estu_pais_reside.png" alt = "Desbalanceo de ESTU_PAIS_RESIDE" width = "100%"></img>

En esta variable, que describe si el estudiante realizó la prueba de estado el día sábado podemos ver un desbalanceo donde el 90.7 % de las ocurrencias son hacia la categoría NO, que se entiende como que el estudiante no realizó la prueba ese día, dentro del análisis y viendo las correlaciones entre esta variable y la variable objetivo (PUNT_GLOBAL), que es de 0.054, también se puede concluir que no sería de mucha utilidad para la realización de un modelo futuro.

<img src = "./resources/images/imbalance_estu_presentacionsabado.png" alt = "Desbalanceo de ESTU_PRESENTACIONSABADO" width = "100%"></img>

Continuando con el análisis; esta variable, que describe si el estudiante pertenece a un colegio mixto, femenino o masculino se puede observar un desbalanceo donde el 97.1 % de las ocurrencias son hacia la categoría _'mixto'_, y que el resto de las ocurrencias (2,9 %) son mínimas y así como en el anterior ejemplo se observa la correlación entre esta variable y la variable objetivo (PUNT_GLOBAL), que es de 0.086, también se puede concluir que no sería de mucha utilidad para la realización de un modelo futuro.

<img src = "./resources/images/imbalance_cole_genero.png" alt = "Desbalanceo de COLE_GENERO" width = "100%"></img>

La siguiente variable describe si el colegio al que el estudiante pertenece está apegado a alguno de los calendarios de año escolar lectivo (A, B, entre otros); en este podemos ver un desbalanceo donde el 99.6 % de las ocurrencias son hacia la categoría A, que se entiende como que el colegio al que el estudiante pertenece está apegado al calendario académico A. Con esta cantidad desorbitada de ocurrencias se entiende que el peso que tienen las ocurrencias de las otras categorías es mínimo y por eso se expone como una variable que no será útil en la fabricación de un modelo a futuro.

<img src = "./resources/images/imbalance_cole_calendario.png" alt = "Desbalanceo de COLE_CALENDARIO" width = "100%"></img>

La variable describe si el colegio al que el estudiante pertenece es o no bilingüe; en este podemos ver un desbalanceo y falta de datos donde el 80.4 % de las ocurrencias son de la categoría N, que se entiende como que el colegio al que el estudiante pertenece no es bilingüe. Además, al tener una cantidad importante de datos faltantes (18.6 %), que hace que esas dos categorías (N y faltante) sume hasta el 99 % de las ocurrencias hace que se concluya que es una variable que no será útil para un modelo en el futuro.

<img src = "./resources/images/imbalance_cole_bilingue.png" alt = "Desbalanceo de COLE_BILINGUE" width = "100%"></img>

Finalizando, la siguiente variable describe si la sede del colegio al que pertenece el estudiante es la sede principal del colegio o no, esta variable en particular tiene un comportamiento extraño teniendo hasta 4 categorías para dos tipos de ocurrencias en total, de cualquier forma se observa que existe un gran desbalanceo con la suma de las dos categorías S sumando hasta el 94.4 % del total de las ocurrencias, haciendo que se entienda esta variable no es o no podría ser importante para un modelo posterior. 

<img src = "./resources/images/imbalance_cole_sede_principal.png" alt = "Desbalanceo de COLE_SEDE_PRINCIPAL dataset" width = "100%"></img>

Por último, esta variable describe si el estudiante realizó la prueba de estado estando privado de la libertad, en ella podemos ver un desbalanceo donde el 90.7 % de las ocurrencias son hacia la categoría N, que se entiende como que el estudiante no estaba privado de la libertad cuando realizó la prueba. Así como en la anterior variable se entiende que con una cantidad tan grande de ocurrencias en la categoría en la que se encuentra el desbalance esta variable no será útil para su uso en un modelo posterior.

<img src = "./resources/images/imbalance_estu_privado_libertad.png" alt = "Desbalanceo de ESTU_PRIVADO_LIBERTAD" width = "100%"></img>


## **Créditos**
* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).
    
**Universidad Nacional de Colombia** - *Facultad de Ingeniería*